In [1]:
import tensorflow as tf
import keras
import time
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

import qkeras
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Model

from callbacks import all_callbacks

2023-06-29 04:12:30.429058: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

cifar10_classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Activation
from qkeras import QActivation, QDense, QConv2D
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.regularizers import l1, l2

# input
input_layer = Input(shape=(32, 32, 3))

# conv1
x = QConv2D(
    8,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='valid',
    name='conv1',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(input_layer)
x = QActivation(activation=quantized_relu(6), name='relu1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool1')(x)

# conv2
x = QConv2D(
    16,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='same',
    name='conv2',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu2')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool2')(x)

# conv3
x = QConv2D(
    16,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv3',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu3')(x)

# conv4
x = QConv2D(
    16,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv4',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu4')(x)

# conv5
x = QConv2D(
    8,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv5',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu5')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool5')(x)

# FC
x = Flatten()(x)
x = QDense(
    16,
    name='fc1',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu6')(x)

x = QDense(
    32,
    name='fc2',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu7')(x)

x = QDense(
    10,
    name='output',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
)(x)
output_layer = Activation(activation='softmax', name='softmax')(x)

# Gene model
model = Model(inputs=input_layer, outputs=output_layer)

model.summary()


2023-06-29 04:12:32.339801: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-29 04:12:32.401942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-29 04:12:32.402264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-29 04:12:32.402285: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-29 04:12:32.404532: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-29 04:12:32.404567: I tensorflow/stream_executor

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1 (QConv2D)              (None, 30, 30, 8)         224       
_________________________________________________________________
relu1 (QActivation)          (None, 30, 30, 8)         0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 14, 14, 8)         0         
_________________________________________________________________
conv2 (QConv2D)              (None, 14, 14, 16)        3216      
_________________________________________________________________
relu2 (QActivation)          (None, 14, 14, 16)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 7, 7, 16)          0     

In [4]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

In [5]:
train = True
if train:
    adam = Adam(lr=0.01)
    sgd = SGD(lr=1)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    
    callbacks = all_callbacks(
        stop_patience=200,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.0001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir='AlexNet_Weights/Qkeras',
    )
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    
    model.fit(
        x_train,
        y_train,
        batch_size=1024,
        epochs=5000,
        validation_data=(x_test, y_test),
        shuffle=True,
        callbacks=callbacks.callbacks,
    )
    
    model = strip_pruning(model)
    model.save('AlexNet_Weights/AlexNet_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('AlexNet_Weights/AlexNet_check_best_model.h5', custom_objects=co)

/home/lab-shi/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2023-06-29 04:12:35.991750: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-29 04:12:35.991766: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-29 04:12:35.991808: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2023-06-29 04:12:35.992460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-29 04:12:35.992546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot 

Epoch 1/5000
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2023-06-29 04:12:41.270916: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:808] layout failed: Invalid argument: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model/prune_low_magnitude_relu5/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-06-29 04:12:41.839325: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-29 04:12:42.285585: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-06-29 04:12:42.818461: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-29 04:12:43.224538: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


 9/49 [====>.........................] - ETA: 1s - loss: 2.3184 - accuracy: 0.1248

2023-06-29 04:12:43.718830: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-29 04:12:43.718853: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-29 04:12:43.718881: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-29 04:12:43.739343: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-06-29 04:12:43.739402: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-29 04:12:43.744963: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-06-29 04:12:43.748157: I tensorflow

...

49/49 [==============================] - 1s 19ms/step - loss: 1.1865 - accuracy: 0.5906 - val_loss: 1.2702 - val_accuracy: 0.5590

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: val_loss did not improve from 1.24756

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

Epoch 00440: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_epoch440.h5

***callbacks end***

Epoch 441/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2021 - accuracy: 0.5832 - val_loss: 1.2719 - val_accuracy: 0.5552

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: val_loss did not improve from 1.24756

Epoch 00441: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00441: saving model to A


***callbacks end***

Epoch 456/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.2028 - accuracy: 0.5859 - val_loss: 1.2973 - val_accuracy: 0.5492

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: val_loss did not improve from 1.24756

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00456: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 457/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1968 - accuracy: 0.5867 - val_loss: 1.2571 - val_accuracy: 0.5620

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: val_loss did not improve from 1.24756

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00457: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w


***callbacks end***

Epoch 472/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1821 - accuracy: 0.5937 - val_loss: 1.2782 - val_accuracy: 0.5547

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: val_loss did not improve from 1.24756

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00472: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 473/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1840 - accuracy: 0.5918 - val_loss: 1.2867 - val_accuracy: 0.5524

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: val_loss did not improve from 1.24756

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00473: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w

49/49 [==============================] - 1s 19ms/step - loss: 1.1838 - accuracy: 0.5919 - val_loss: 1.2540 - val_accuracy: 0.5625

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: val_loss did not improve from 1.24756

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00488: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 489/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1911 - accuracy: 0.5907 - val_loss: 1.2702 - val_accuracy: 0.5588

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: val_loss did not improve from 1.24756

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00489: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch


***callbacks end***

Epoch 504/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1937 - accuracy: 0.5890 - val_loss: 1.2611 - val_accuracy: 0.5609

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: val_loss did not improve from 1.24756

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00504: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 505/5000
49/49 [==============================] - 1s 19ms/step - loss: 1.1883 - accuracy: 0.5891 - val_loss: 1.2524 - val_accuracy: 0.5650

***callbacks***
saving losses to AlexNet_Weights/Qkeras/losses.log

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: val_loss did not improve from 1.24756

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last.h5

Epoch 00505: saving model to AlexNet_Weights/Qkeras/KERAS_check_model_last_w

In [6]:
predictions = model.predict(x_test)
predictions

array([[2.66918577e-02, 2.03259345e-02, 9.37569886e-02, ...,
        1.77372526e-02, 7.25205764e-02, 1.06017459e-02],
       [1.31269932e-01, 2.38280177e-01, 5.72271459e-03, ...,
        1.26530752e-02, 2.96399713e-01, 2.76679873e-01],
       [2.13042498e-01, 2.26893321e-01, 1.17750624e-02, ...,
        1.10562472e-02, 4.32464182e-01, 6.34381846e-02],
       ...,
       [9.54090524e-03, 2.64941365e-03, 1.35425314e-01, ...,
        2.36636639e-01, 1.29627471e-03, 6.26319135e-03],
       [1.09956056e-01, 3.82661581e-01, 1.23265259e-01, ...,
        7.32107610e-02, 3.03804269e-03, 3.25347036e-02],
       [2.92310165e-03, 5.80477249e-03, 2.67454190e-03, ...,
        8.89720142e-01, 2.02545983e-04, 8.20204522e-03]], dtype=float32)